In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os

# Đọc dữ liệu
print("Đang đọc dữ liệu...")
csv_files = [f for f in os.listdir("weather_archive") if f.endswith('.csv')]
if not csv_files:
    raise FileNotFoundError("Không tìm thấy file CSV trong thư mục weather_archive")

data_file = os.path.join("weather_archive", csv_files[0])
df = pd.read_csv(data_file)

# Kiểm tra và làm sạch dữ liệu
print("Đang xử lý dữ liệu...")
df['Date'] = pd.to_datetime(df['Date'])
df = df.dropna(subset=['Temp_Max', 'Temp_Min', 'Weather_Code'])
df['Weather_Code'] = df['Weather_Code'].astype(int)

# Định nghĩa các mã thời tiết hợp lệ và mô tả
valid_weather_codes = [0, 1, 2, 3, 45, 48, 51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 71, 73, 75, 77, 80, 81, 82, 85, 86, 95, 96, 99]
weather_code_descriptions = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    45: "Fog",
    48: "Depositing rime fog",
    51: "Drizzle: Light intensity",
    53: "Drizzle: Moderate intensity",
    55: "Drizzle: Dense intensity",
    56: "Freezing drizzle: Light",
    57: "Freezing drizzle: Dense",
    61: "Rain: Slight",
    63: "Rain: Moderate",
    65: "Rain: Heavy",
    66: "Freezing rain: Light",
    67: "Freezing rain: Heavy",
    71: "Snow fall: Slight",
    73: "Snow fall: Moderate",
    75: "Snow fall: Heavy",
    77: "Snow grains",
    80: "Rain showers: Slight",
    81: "Rain showers: Moderate",
    82: "Rain showers: Violent",
    85: "Snow showers: Slight",
    86: "Snow showers: Heavy",
    95: "Thunderstorm: Slight or moderate",
    96: "Thunderstorm with slight hail",
    99: "Thunderstorm with heavy hail"
}

# Hàm tìm mã thời tiết gần nhất
def find_nearest_weather_code(pred_code, valid_codes=valid_weather_codes):
    valid_codes = np.array(valid_codes)
    idx = (np.abs(valid_codes - pred_code)).argmin()
    return valid_codes[idx]

# Thêm các đặc trưng thời gian
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfYear'] = df['Date'].dt.dayofyear
df['DayOfWeek'] = df['Date'].dt.dayofweek

# Mã hóa tỉnh thành
le_province = LabelEncoder()
df['Province_Code'] = le_province.fit_transform(df['Province'])

# Tạo thư mục lưu kết quả dự đoán
os.makedirs('xgboost_predictions', exist_ok=True)

# Lấy danh sách các tỉnh thành
provinces = df['Province'].unique()

print(f"\n{'='*50}")
print(f"ĐÁNH GIÁ MÔ HÌNH CHO {len(provinces)} TỈNH THÀNH")
print(f"{'='*50}")

# Tạo DataFrame để lưu kết quả dự đoán
prediction_results = pd.DataFrame()

for province_name in provinces:
    print(f"\nĐang xử lý cho tỉnh: {province_name}")
    
    # Lọc dữ liệu theo tỉnh
    province_data = df[df['Province'] == province_name].sort_values('Date')
    
    # Kiểm tra xem có đủ dữ liệu không
    if len(province_data) < 14:
        print(f"  Bỏ qua tỉnh {province_name}: không đủ dữ liệu")
        continue
    
    # Tạo các đặc trưng với độ trễ
    for lag in range(1, 8):
        province_data[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].shift(lag)
        province_data[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].shift(lag)
        province_data[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].shift(lag)
    
    # Tạo đặc trưng trung bình động
    for window in [3, 5, 7]:
        province_data[f'Temp_Max_Roll{window}'] = province_data['Temp_Max'].rolling(window=window).mean()
        province_data[f'Temp_Min_Roll{window}'] = province_data['Temp_Min'].rolling(window=window).mean()
        province_data[f'Weather_Code_Roll{window}'] = province_data['Weather_Code'].rolling(window=window).mean()
    
    # Loại bỏ các hàng có giá trị NaN
    province_data = province_data.dropna()
    
    if len(province_data) < 10:
        print(f"  Bỏ qua tỉnh {province_name}: không đủ dữ liệu sau khi tạo đặc trưng")
        continue
    
    # Chia tập dữ liệu train/test (80/20)
    train_size = int(len(province_data) * 0.8)
    train_data = province_data.iloc[:train_size]
    test_data = province_data.iloc[train_size:]
    
    # Tạo các đặc trưng cho dự đoán
    feature_cols = [col for col in train_data.columns if col.startswith(('Temp_Max_Lag', 'Temp_Min_Lag', 
                                                                        'Weather_Code_Lag', 'Temp_Max_Roll', 
                                                                        'Temp_Min_Roll', 'Weather_Code_Roll'))]
    feature_cols += ['Month', 'Day', 'DayOfYear', 'DayOfWeek']
    
    X_train = train_data[feature_cols]
    X_test = test_data[feature_cols]
    y_train_temp_max = train_data['Temp_Max']
    y_test_temp_max = test_data['Temp_Max']
    y_train_temp_min = train_data['Temp_Min']
    y_test_temp_min = test_data['Temp_Min']
    y_train_weather_code = train_data['Weather_Code']
    y_test_weather_code = test_data['Weather_Code']
    
    # Huấn luyện mô hình
    models = {
        'temp_max': xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
        'temp_min': xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
        'weather_code': xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
    }
    
    # Huấn luyện và đánh giá từng mô hình
    print(f"\nKết quả RMSE cho {province_name}:")
    print("-" * 40)
    
    for target, model in models.items():
        if target == 'temp_max':
            y_train = y_train_temp_max
            y_test = y_test_temp_max
            target_name = "Nhiệt độ cao nhất"
        elif target == 'temp_min':
            y_train = y_train_temp_min
            y_test = y_test_temp_min
            target_name = "Nhiệt độ thấp nhất"
        else:
            y_train = y_train_weather_code
            y_test = y_test_weather_code
            target_name = "Mã thời tiết"
        
        # Huấn luyện mô hình
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Tính RMSE
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        # In kết quả
        print(f"{target_name}: {rmse:.4f}")
    
    print("-" * 40)
    
    # Dự đoán cho 7 ngày tiếp theo
    last_date = province_data['Date'].max()
    future_dates = [last_date + timedelta(days=i+1) for i in range(7)]
    
    # Tạo DataFrame để lưu kết quả dự đoán cho tỉnh này
    province_predictions = pd.DataFrame()
    
    # Lấy dữ liệu gần nhất để dự đoán
    last_row = province_data.iloc[-1].copy()
    
    for i in range(7):
        future_date = future_dates[i]
        predict_row = pd.Series(dtype='float64')
        
        # Thêm đặc trưng thời gian
        predict_row['Month'] = future_date.month
        predict_row['Day'] = future_date.day
        predict_row['DayOfYear'] = future_date.dayofyear
        predict_row['DayOfWeek'] = future_date.dayofweek
        
        # Cập nhật lag features
        if i == 0:
            for lag in range(1, 8):
                if lag <= len(province_data):
                    predict_row[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].iloc[-lag]
                    predict_row[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].iloc[-lag]
                    predict_row[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].iloc[-lag]
                else:
                    predict_row[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].mean()
                    predict_row[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].mean()
                    predict_row[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].mean()
        else:
            for lag in range(1, 8):
                if lag <= i:
                    predict_row[f'Temp_Max_Lag{lag}'] = province_predictions['Temp_Max'].iloc[i-lag]
                    predict_row[f'Temp_Min_Lag{lag}'] = province_predictions['Temp_Min'].iloc[i-lag]
                    predict_row[f'Weather_Code_Lag{lag}'] = province_predictions['Weather_Code'].iloc[i-lag]
                else:
                    idx = lag - i - 1
                    if idx < len(province_data):
                        predict_row[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].iloc[-(lag-i)]
                        predict_row[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].iloc[-(lag-i)]
                        predict_row[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].iloc[-(lag-i)]
                    else:
                        predict_row[f'Temp_Max_Lag{lag}'] = province_data['Temp_Max'].mean()
                        predict_row[f'Temp_Min_Lag{lag}'] = province_data['Temp_Min'].mean()
                        predict_row[f'Weather_Code_Lag{lag}'] = province_data['Weather_Code'].mean()
        
        # Cập nhật rolling mean
        for window in [3, 5, 7]:
            if window <= i + 1:
                temp_max_vals = []
                temp_min_vals = []
                weather_code_vals = []
                
                for w in range(window):
                    if w < i:
                        temp_max_vals.append(province_predictions['Temp_Max'].iloc[i-w-1])
                        temp_min_vals.append(province_predictions['Temp_Min'].iloc[i-w-1])
                        weather_code_vals.append(province_predictions['Weather_Code'].iloc[i-w-1])
                    else:
                        idx = w - i
                        if idx < len(province_data):
                            temp_max_vals.append(province_data['Temp_Max'].iloc[-idx-1])
                            temp_min_vals.append(province_data['Temp_Min'].iloc[-idx-1])
                            weather_code_vals.append(province_data['Weather_Code'].iloc[-idx-1])
                        else:
                            temp_max_vals.append(province_data['Temp_Max'].mean())
                            temp_min_vals.append(province_data['Temp_Min'].mean())
                            weather_code_vals.append(province_data['Weather_Code'].mean())
                
                predict_row[f'Temp_Max_Roll{window}'] = np.mean(temp_max_vals)
                predict_row[f'Temp_Min_Roll{window}'] = np.mean(temp_min_vals)
                predict_row[f'Weather_Code_Roll{window}'] = np.mean(weather_code_vals)
            else:
                predict_row[f'Temp_Max_Roll{window}'] = province_data['Temp_Max'].mean()
                predict_row[f'Temp_Min_Roll{window}'] = province_data['Temp_Min'].mean()
                predict_row[f'Weather_Code_Roll{window}'] = province_data['Weather_Code'].mean()
        
        # Chuyển đổi Series thành DataFrame
        predict_df = pd.DataFrame([predict_row])
        
        # Đảm bảo tất cả các đặc trưng đều có
        for col in feature_cols:
            if col not in predict_df.columns:
                predict_df[col] = 0
        
        # Chỉ giữ lại các đặc trưng trong mô hình
        predict_df = predict_df[feature_cols]
        
        # Dự đoán
        temp_max_pred = models['temp_max'].predict(predict_df)[0]
        temp_min_pred = models['temp_min'].predict(predict_df)[0]
        weather_code_pred_raw = models['weather_code'].predict(predict_df)[0]
        weather_code_pred = find_nearest_weather_code(weather_code_pred_raw)
        
        # Thêm vào DataFrame dự đoán
        province_predictions = pd.concat([province_predictions, pd.DataFrame({
            'Date': [future_date],
            'Temp_Max': [temp_max_pred],
            'Temp_Min': [temp_min_pred],
            'Weather_Code': [weather_code_pred],
            'Weather_Description': [weather_code_descriptions[weather_code_pred]]
        })])
    
    # Thêm cột Province
    province_predictions['Province'] = province_name
    
    # Thêm vào kết quả tổng thể
    prediction_results = pd.concat([prediction_results, province_predictions])

# Lưu kết quả dự đoán
prediction_results = prediction_results[['Province', 'Date', 'Temp_Max', 'Temp_Min', 'Weather_Code', 'Weather_Description']]
prediction_results.to_csv('xgboost_predictions/weather_forecast_7days.csv', index=False)

print(f"\n{'='*50}")
print("ĐÃ HOÀN THÀNH DỰ BÁO THỜI TIẾT!")
print(f"Kết quả được lưu trong file: 'xgboost_predictions/weather_forecast_7days.csv'")
print(f"{'='*50}")

Đang đọc dữ liệu...
Đang xử lý dữ liệu...

ĐÁNH GIÁ MÔ HÌNH CHO 63 TỈNH THÀNH

Đang xử lý cho tỉnh: An Giang

Kết quả RMSE cho An Giang:
----------------------------------------
Nhiệt độ cao nhất: 2.4033
Nhiệt độ thấp nhất: 0.6676
Mã thời tiết: 21.1277
----------------------------------------

Đang xử lý cho tỉnh: Bà Rịa - Vũng Tàu

Kết quả RMSE cho Bà Rịa - Vũng Tàu:
----------------------------------------
Nhiệt độ cao nhất: 1.5396
Nhiệt độ thấp nhất: 1.2686
Mã thời tiết: 30.1011
----------------------------------------

Đang xử lý cho tỉnh: Bạc Liêu

Kết quả RMSE cho Bạc Liêu:
----------------------------------------
Nhiệt độ cao nhất: 1.7672
Nhiệt độ thấp nhất: 0.8460
Mã thời tiết: 9.2727
----------------------------------------

Đang xử lý cho tỉnh: Bắc Giang

Kết quả RMSE cho Bắc Giang:
----------------------------------------
Nhiệt độ cao nhất: 1.5951
Nhiệt độ thấp nhất: 2.7817
Mã thời tiết: 26.4588
----------------------------------------

Đang xử lý cho tỉnh: Bắc Kạn

Kết quả 